In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
df_60s = pd.read_csv('/content/drive/MyDrive/quickview of price surge in nigeria/1960_onwards.csv')

# Display the first 5 rows
print(df_60s.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df_60s.info())


| Year   | Consumer price index (2010 = 100)   | GDP (constant LCU)   | GDP (current LCU)   | GDP growth (annual %)   | GDP per capita (current US$)   | GDP per capita growth (annual %)   | Inflation, GDP deflator (annual %)   | Inflation, consumer prices (annual %)   | Livestock production index (2014-2016 = 100)   | Official exchange rate (LCU per US$, period average)   | Total reserves (includes gold, current US$)   | Population, total   | Population, female   | Population, male   | Population ages 0-14 (% of total population)   | Population ages 15-64 (% of total population)   | Population ages 65 and above (% of total population)   | Trade Openness Index(%)   | Crude oil price(per barrel in $)   | Cumulative crude oil production up to and including year   | Narrow Money   | Money Supply M3   | Net Foreign Assets   | Net Domestic Credit   | Credit to Government   | Credit to Private Sector   | Base Money   | Currency in Circulation   | Bank Reserves   | Currency Outside Banks   | D

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

# 1. Read the CSV file into a DataFrame
#df_60s = pd.read_csv('/content/drive/MyDrive/quickview of price surge in nigeria/1960_onwards.csv')

# 2. Convert the 'Year' column to datetime and set it as the index
df_60s['Year'] = pd.to_datetime(df_60s['Year'], format='%Y')
df_60s.set_index('Year', inplace=True)

# 3. Create a SARIMAX model with order (1, 1, 1) and seasonal order (1, 1, 1, 12)
model = SARIMAX(df_60s['GDP growth (annual %)'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# 4. Fit the SARIMAX model to the 'GDP growth (annual %)' data
results = model.fit()

# 5. Generate predictions for the next 10 years
forecast = results.get_forecast(steps=10)

# 6. Print the forecasted values rounded to 2 decimal places.
print("Forecasted GDP growth (annual %):")
print(forecast.predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 7. Create a Plotly line plot to visualize the original and forecasted 'GDP growth (annual %)'
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_60s.index, y=df_60s['GDP growth (annual %)'],
                    mode='lines+markers',
                    name='Actual GDP Growth'))
fig.add_trace(go.Scatter(x=forecast.predicted_mean.index, y=forecast.predicted_mean,
                    mode='lines+markers',
                    name='Forecasted GDP Growth'))

# 8. Add a vertical line to indicate the start of the forecast period
fig.add_vline(x=forecast.predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 9. Label the axes and add a title.
fig.update_layout(title='GDP Growth (Annual %) - Actual vs. Forecast',
                   xaxis_title='Year',
                   yaxis_title='GDP Growth (annual %)')

# 10. Display the plot
fig.show()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


Forecasted GDP growth (annual %):
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 3.64             |
| 2026-01-01 00:00:00 | 4.43             |
| 2027-01-01 00:00:00 | 0.55             |
| 2028-01-01 00:00:00 | 4.37             |
| 2029-01-01 00:00:00 | 3.55             |
| 2030-01-01 00:00:00 | 5.29             |
| 2031-01-01 00:00:00 | 2.57             |
| 2032-01-01 00:00:00 | 3.3              |
| 2033-01-01 00:00:00 | 5.28             |
| 2034-01-01 00:00:00 | 5.85             |


SAVE MODEL AS PICKLE


In [ ]:
import pickle

# Save the series to a pickle file
with open('gdp_growth.pkl', 'wb') as f:
    pickle.dump(gdp_growth_series, f)

print("GDP growth series saved to gdp_growth.pkl")


In [ ]:
import altair as alt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# No need to convert 'Year' to datetime again, it's already the index

# Create a new dataframe df by copying df_60s to avoid modifying the original dataframe
df = df_60s.copy()

# 'Year' is already the index, no need to set it again

# Create a SARIMAX model with order (1, 1, 1) and seasonal order (1, 1, 1, 12)
model = SARIMAX(df['GDP growth (annual %)'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the SARIMAX model to the 'GDP growth (annual %)' data
results = model.fit()

# Generate predictions for the next 10 years
forecast = results.get_forecast(steps=10)

# Print the forecasted values
print("Forecasted GDP growth (annual %):")
print(forecast.predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# Reset the index before plotting
# to have 'Year' as a column for Altair
df_reset = df['GDP growth (annual %)'].reset_index()
forecast_reset = forecast.predicted_mean.reset_index().rename(columns={0: 'GDP growth (annual %)'})

# Create a plot to visualize the original and forecasted 'GDP growth (annual %)'
chart = alt.Chart(pd.concat([df_reset, forecast_reset])).mark_line(point=True).encode(
    x='Year:T',
    y='GDP growth (annual %):Q',
    color=alt.Color('index:N', scale=alt.Scale(domain=['GDP growth (annual %)'], range=['blue', 'red'])),
    tooltip=['Year', 'GDP growth (annual %)']
).properties(
    title='GDP Growth (Annual %) - Actual vs. Forecast'
).interactive()

# Save the chart
chart.save('gdp_growth_forecast_sarimax.json')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning:

Non-invertible starting seasonal moving average Using zeros as starting parameters.



Forecasted GDP growth (annual %):
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 3.64             |
| 2026-01-01 00:00:00 | 4.43             |
| 2027-01-01 00:00:00 | 0.55             |
| 2028-01-01 00:00:00 | 4.37             |
| 2029-01-01 00:00:00 | 3.55             |
| 2030-01-01 00:00:00 | 5.29             |
| 2031-01-01 00:00:00 | 2.57             |
| 2032-01-01 00:00:00 | 3.3              |
| 2033-01-01 00:00:00 | 5.28             |
| 2034-01-01 00:00:00 | 5.85             |


In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

# 1. Read the CSV file into a DataFrame
df_60s = pd.read_csv('/content/drive/MyDrive/quickview of price surge in nigeria/data Vizualizations /1960_onwards (1).csv')

# 2. Convert the 'Year' column to datetime and set it as the index
df_60s['Year'] = pd.to_datetime(df_60s['Year'], format='%Y')
df_60s.set_index('Year', inplace=True)

# 3. Create a SARIMAX model for 'Petrol Price (Naira)'
model_petrol = SARIMAX(df_60s['Petrol Price (Naira)'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# 4. Fit the SARIMAX model to the 'Petrol Price (Naira)' data
results_petrol = model_petrol.fit()

# 5. Generate predictions for the next 10 years for 'Petrol Price (Naira)'
forecast_petrol = results_petrol.get_forecast(steps=10)

# 6. Print the forecasted values for 'Petrol Price (Naira)'
print("Forecasted Petrol Price (Naira):")
print(forecast_petrol.predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 7. Create a SARIMAX model for 'Population, total'
model_population = SARIMAX(df_60s['Population, total'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# 8. Fit the SARIMAX model to the 'Population, total' data
results_population = model_population.fit()

# 9. Generate predictions for the next 10 years for 'Population, total'
forecast_population = results_population.get_forecast(steps=10)

# 10. Print the forecasted values for 'Population, total'
print("\nForecasted Population, total:")
print(forecast_population.predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 11. Create a Plotly line plot to visualize the actual and forecasted values
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_60s.index, y=df_60s['Petrol Price (Naira)'],
                    mode='lines+markers',
                    name='Actual Petrol Price'))
fig.add_trace(go.Scatter(x=forecast_petrol.predicted_mean.index, y=forecast_petrol.predicted_mean,
                    mode='lines+markers',
                    name='Forecasted Petrol Price'))
fig.add_trace(go.Scatter(x=df_60s.index, y=df_60s['Population, total'],
                    mode='lines+markers',
                    name='Actual Population'))
fig.add_trace(go.Scatter(x=forecast_population.predicted_mean.index, y=forecast_population.predicted_mean,
                    mode='lines+markers',
                    name='Forecasted Population'))

# 12. Add a vertical line to indicate the start of the forecast period
fig.add_vline(x=forecast_petrol.predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 13. Label the axes and add a title
fig.update_layout(title='Petrol Price (Naira) and Population, total - Actual vs. Forecast',
                   xaxis_title='Year',
                   yaxis_title='Value')

# 14. Display the plot
fig.show()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning:

Non-stationary starting seasonal autoregressive Using zeros as starting parameters.



Forecasted Petrol Price (Naira):
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 700.67           |
| 2026-01-01 00:00:00 | 725.23           |
| 2027-01-01 00:00:00 | 770.93           |
| 2028-01-01 00:00:00 | 795.26           |
| 2029-01-01 00:00:00 | 821.05           |
| 2030-01-01 00:00:00 | 837.6            |
| 2031-01-01 00:00:00 | 857.51           |
| 2032-01-01 00:00:00 | 891.82           |
| 2033-01-01 00:00:00 | 911.77           |
| 2034-01-01 00:00:00 | 951.28           |


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.




Forecasted Population, total:
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 2.2391e+08       |
| 2026-01-01 00:00:00 | 2.29282e+08      |
| 2027-01-01 00:00:00 | 2.34616e+08      |
| 2028-01-01 00:00:00 | 2.40008e+08      |
| 2029-01-01 00:00:00 | 2.45572e+08      |
| 2030-01-01 00:00:00 | 2.51203e+08      |
| 2031-01-01 00:00:00 | 2.56862e+08      |
| 2032-01-01 00:00:00 | 2.62568e+08      |
| 2033-01-01 00:00:00 | 2.68335e+08      |
| 2034-01-01 00:00:00 | 2.74174e+08      |


In [ ]:
import pickle

# Save the 'Petrol Price (Naira)' model
with open('sarimax_model_petrol.pkl', 'wb') as pkl_file:
    pickle.dump(results_petrol, pkl_file)

# Save the 'Population, total' model
with open('sarimax_model_population.pkl', 'wb') as pkl_file:
    pickle.dump(results_population, pkl_file)

print("Models saved successfully!")



In [ ]:
# Load the 'Petrol Price (Naira)' model
with open('sarimax_model_petrol.pkl', 'rb') as pkl_file:
    loaded_model_petrol = pickle.load(pkl_file)

# Load the 'Population, total' model
with open('sarimax_model_population.pkl', 'rb') as pkl_file:
    loaded_model_population = pickle.load(pkl_file)

# Generate new forecasts (example)
new_forecast_petrol = loaded_model_petrol.get_forecast(steps=5)
new_forecast_population = loaded_model_population.get_forecast(steps=5)


PETROL PRICE


In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.express as px

# 1. Read the CSV file into a DataFrame
df_60s = pd.read_csv('/content/drive/MyDrive/quickview of price surge in nigeria/data Vizualizations /1960_onwards (1).csv')

# 2. Convert the 'Year' column to datetime and set it as the index
df_60s['Year'] = pd.to_datetime(df_60s['Year'], format='%Y')
df_60s.set_index('Year', inplace=True)

# 3. Create a SARIMAX model for 'Petrol Price (Naira)'
model_petrol = SARIMAX(df_60s['Petrol Price (Naira)'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# 4. Fit the SARIMAX model to the 'Petrol Price (Naira)' data
results_petrol = model_petrol.fit()

# 5. Generate predictions for the next 10 years for 'Petrol Price (Naira)'
forecast_petrol = results_petrol.get_forecast(steps=10)

# 6. Print the forecasted values for 'Petrol Price (Naira)'
print("Forecasted Petrol Price (Naira):")
print(forecast_petrol.predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 7. Create a Plotly Express line plot to visualize the actual and forecasted values for 'Petrol Price (Naira)'
fig = px.line(df_60s, x=df_60s.index, y='Petrol Price (Naira)', title='Petrol Price (Naira) - Actual vs. Forecast')

# Add the forecasted values to the plot
fig.add_scatter(x=forecast_petrol.predicted_mean.index, y=forecast_petrol.predicted_mean, mode='lines+markers', name='Forecast')

# 8. Add a vertical line to indicate the start of the forecast period
fig.add_vline(x=forecast_petrol.predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 9. Label the axes and add a title
fig.update_layout(xaxis_title='Year', yaxis_title='Petrol Price (Naira)')

# 10. Display the plot
fig.show()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning:

Non-stationary starting seasonal autoregressive Using zeros as starting parameters.



Forecasted Petrol Price (Naira):
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 700.67           |
| 2026-01-01 00:00:00 | 725.23           |
| 2027-01-01 00:00:00 | 770.93           |
| 2028-01-01 00:00:00 | 795.26           |
| 2029-01-01 00:00:00 | 821.05           |
| 2030-01-01 00:00:00 | 837.6            |
| 2031-01-01 00:00:00 | 857.51           |
| 2032-01-01 00:00:00 | 891.82           |
| 2033-01-01 00:00:00 | 911.77           |
| 2034-01-01 00:00:00 | 951.28           |


In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
import altair as alt

# 1. Read the CSV file into a DataFrame
#df_60s = pd.read_csv('1960_onwards (1).csv')


# 2. Convert the 'Year' column to datetime and set it as the index (only if 'Year' is still a column)
if 'Year' in df_60s.columns:
    df_60s['Year'] = pd.to_datetime(df_60s['Year'], format='%Y')
    df_60s.set_index('Year', inplace=True)


# 3. Create a list of columns to forecast
columns_to_forecast = ['Population, total', 'Population, female', 'Population, male',
                       'Population ages 0-14 (% of total population)',
                       'Population ages 15-64 (% of total population)',
                       'Population ages 65 and above (% of total population)']

# 4. Create dictionaries to store models and forecasts
models = {}
forecasts = {}

# 5. Iterate through the columns and build SARIMAX models
for column in columns_to_forecast:
    model = SARIMAX(df_60s[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()
    models[column] = results
    forecasts[column] = results.get_forecast(steps=10)

# 6. Print the forecasted values for each column
for column in columns_to_forecast:
    print(f"\nForecasted {column}:")
    print(forecasts[column].predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 7. Create a Plotly figure with subplots
fig = go.Figure()
for i, column in enumerate(columns_to_forecast):
    # 8. Add traces for actual and forecasted values
    fig.add_trace(go.Scatter(x=df_60s.index, y=df_60s[column],
                            mode='lines+markers',
                            name=f'Actual {column}'))
    fig.add_trace(go.Scatter(x=forecasts[column].predicted_mean.index, y=forecasts[column].predicted_mean,
                            mode='lines+markers',
                            name=f'Forecasted {column}'))

    # 9. Add a vertical line to indicate the start of the forecast period
    fig.add_vline(x=forecasts[column].predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 10. Update layout
fig.update_layout(title='Population Forecasts (Total, Male, Female, Age Groups)',
                  xaxis_title='Year',
                  yaxis_title='Value')

# 11. Display the plot
fig.show()

# Create Altair charts for each population forecast
for column in columns_to_forecast:
    chart_data = pd.concat([
        df_60s[column].reset_index().rename(columns={'index': 'Year'}),
        forecasts[column].predicted_mean.reset_index().rename(columns={'index': 'Year', 0: column})
    ])

    chart_data['Data Type'] = ['Actual'] * len(df_60s) + ['Forecast'] * len(forecasts[column].predicted_mean)

    chart = alt.Chart(chart_data).mark_line(point=True).encode(
        x='Year:T',
        y=f'{column}:Q',
        color='Data Type:N',
        tooltip=['Year', column, 'Data Type']
    ).properties(
        title=f'{column} - Actual vs. Forecast'
    ).interactive()

    # Save the chart
    chart.save(f'{column}_forecast_sarimax.json')


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency informat


Forecasted Population, total:
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 2.2391e+08       |
| 2026-01-01 00:00:00 | 2.29282e+08      |
| 2027-01-01 00:00:00 | 2.34616e+08      |
| 2028-01-01 00:00:00 | 2.40008e+08      |
| 2029-01-01 00:00:00 | 2.45572e+08      |
| 2030-01-01 00:00:00 | 2.51203e+08      |
| 2031-01-01 00:00:00 | 2.56862e+08      |
| 2032-01-01 00:00:00 | 2.62568e+08      |
| 2033-01-01 00:00:00 | 2.68335e+08      |
| 2034-01-01 00:00:00 | 2.74174e+08      |

Forecasted Population, female:
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 1.10321e+08      |
| 2026-01-01 00:00:00 | 1.12587e+08      |
| 2027-01-01 00:00:00 | 1.14844e+08      |
| 2028-01-01 00:00:00 | 1.17124e+08      |
| 2029-01-01 00:00:00 | 1.19481e+08      |
| 2030-01-01 00:00:00 | 1.21869e+08      |
| 2031-01-01 00:00:00 | 1.2427e+08       |
| 2032-01-01 00:00:00 | 1.2673e+08

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



FORECAST FOR POPULATION TOTAL, MALE, FEMALE AND 0-14.

In [ ]:
# 1. Read the CSV file into a DataFrame
#df_60s = pd.read_csv('1960_onwards (1).csv')

# 2. Convert the 'Year' column to datetime and set it as the index
#df_60s['Year'] = pd.to_datetime(df_60s['Year'], format='%Y')
#df_60s.set_index('Year', inplace=True)

# 3. Create a dictionary to store the column names and their corresponding SARIMAX models and forecasts
models_and_forecasts = {}

# 4. Iterate through the columns and build SARIMAX models
for column in ['Population, total', 'Population, female', 'Population, male', 'Population ages 0-14 (% of total population)']:
    model = SARIMAX(df_60s[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()
    forecast = results.get_forecast(steps=10)
    models_and_forecasts[column] = {'model': results, 'forecast': forecast}

# 5. Print the forecasted values for each column
for column, data in models_and_forecasts.items():
    print(f"\nForecasted {column}:")
    print(data['forecast'].predicted_mean.round(2).to_markdown(numalign="left", stralign="left"))

# 6. Create a Plotly line plot to visualize the actual and forecasted values for each column
fig = go.Figure()

# 7. Add traces for actual and forecasted values for each column to the plot
for column, data in models_and_forecasts.items():
    fig.add_trace(go.Scatter(x=df_60s.index, y=df_60s[column],
                    mode='lines+markers',
                    name=f'Actual {column}'))
    fig.add_trace(go.Scatter(x=data['forecast'].predicted_mean.index, y=data['forecast'].predicted_mean,
                    mode='lines+markers',
                    name=f'Forecasted {column}'))

# 8. Add a vertical line to indicate the start of the forecast period
fig.add_vline(x=forecast.predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 9. Label the axes and add a title
fig.update_layout(title='Population Forecasts - Actual vs. Forecast',
                   xaxis_title='Year',
                   yaxis_title='Value')

# 10. Display the plot
fig.show()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency AS-JAN will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency informat


Forecasted Population, total:
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 2.2391e+08       |
| 2026-01-01 00:00:00 | 2.29282e+08      |
| 2027-01-01 00:00:00 | 2.34616e+08      |
| 2028-01-01 00:00:00 | 2.40008e+08      |
| 2029-01-01 00:00:00 | 2.45572e+08      |
| 2030-01-01 00:00:00 | 2.51203e+08      |
| 2031-01-01 00:00:00 | 2.56862e+08      |
| 2032-01-01 00:00:00 | 2.62568e+08      |
| 2033-01-01 00:00:00 | 2.68335e+08      |
| 2034-01-01 00:00:00 | 2.74174e+08      |

Forecasted Population, female:
|                     | predicted_mean   |
|:--------------------|:-----------------|
| 2025-01-01 00:00:00 | 1.10321e+08      |
| 2026-01-01 00:00:00 | 1.12587e+08      |
| 2027-01-01 00:00:00 | 1.14844e+08      |
| 2028-01-01 00:00:00 | 1.17124e+08      |
| 2029-01-01 00:00:00 | 1.19481e+08      |
| 2030-01-01 00:00:00 | 1.21869e+08      |
| 2031-01-01 00:00:00 | 1.2427e+08       |
| 2032-01-01 00:00:00 | 1.2673e+08

In [ ]:
import plotly.graph_objects as go

# 1. Convert the `Forecasted Petrol Price (Naira)` column in `forecast_df` to numeric
forecast_df['Forecasted Petrol Price (Naira)'] = pd.to_numeric(forecast_df['Forecasted Petrol Price (Naira)'], errors='coerce')

# 2. Create a Plotly line plot to visualize the actual 'Petrol Price (Naira)' from `df_augmented`
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_augmented.index, y=df_augmented['Petrol Price (Naira)'],
                    mode='lines+markers',
                    name='Actual Petrol Price'))

# 3. Add a trace for the forecasted 'Petrol Price (Naira)' from `forecast_df` to the plot
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Forecasted Petrol Price (Naira)'],
                    mode='lines+markers',
                    name='Forecasted Petrol Price'))

# 4. Add a vertical line to indicate the start of the forecast period
fig.add_vline(x=forecast_petrol.predicted_mean.index[0], line_width=2, line_dash="dash", line_color="green")

# 5. Label the axes and add a title
fig.update_layout(title='Petrol Price (Naira) - Actual vs. Forecast (Augmented)',
                   xaxis_title='Year',
                   yaxis_title='Petrol Price (Naira)')

# 6. Display the plot
fig.show()
